In [9]:
import pandas as pd
import numpy as np

from influxdb_client import InfluxDBClient, Point, Dialect

import re
import time
import datetime

import warnings
from influxdb_client.client.warnings import MissingPivotFunction

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.colors as colors

import pandasql as ps
import sqlite3

import csv

import warnings 
from influxdb_client.client.warnings import MissingPivotFunction
warnings.simplefilter("ignore", MissingPivotFunction)

import gc

pd.set_option('display.max_rows', 500)

In [12]:
def experiment(start_time, end_time, filepath):
    # Retrieve experiments data from csv
    data = pd.read_csv(filepath, header=None)
    df = pd.DataFrame(data)

    #Rename columns
    experiments = df.rename(columns={0: "start", 1: "end", 2: "topology", 3: "runtime", 4: "parameter", 5: "d", 6: "dlo", 7: "dhi", 8: "dscore", 9: "dlazy", 10: "dout", 11: "gossipFactor", 12: "initialDelay", 13: "interval"}, errors='raise')

    #Correct timestamp
    experiments["start"] = experiments["start"].str.slice(0, 27)
    experiments["end"] = experiments["end"].str.slice(0, 27)

    #String to timestamp
    # experiments['startUnix'] = pd.to_datetime(experiments["start"],format="%Y-%m-%d %H:%M:%S.%f").astype('int64') / 10**9
    # experiments['endUnix'] = pd.to_datetime(experiments["end"],format="%Y-%m-%d %H:%M:%S.%f").astype('int64') / 10**9
    experiments['startUnix'] = pd.to_datetime(experiments["start"],format="mixed").astype('int64') / 10**9
    experiments['endUnix'] = pd.to_datetime(experiments["end"],format="mixed").astype('int64') / 10**9


    experiments['startUnix'] = pd.to_timedelta(experiments['startUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)
    experiments['endUnix'] = pd.to_timedelta(experiments['endUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)

    #Drop fields we don't mneed for the moment
    exp = experiments.drop(columns=["runtime", "initialDelay"]).sort_values(by=["start"])

    #Get times for different intervals
    # intervals = exp["interval"].drop_duplicates().sort_values().reset_index(drop=True)
    # intervals.head(10)

    # expTime = exp[exp['startUnix'].astype(int).between(start_time, end_time)]
    # expTime['experiment'] = expTime.index
    expTime = exp.reset_index().rename({'index':'experiment'}, axis = 'columns')

    return expTime

experiments = experiment(start, end, '../../experiments.csv')
experiments.tail(50)

,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix
616,616,2023-11-10 23:41:01.0448180,2023-11-11 00:11:01.0479786,general,random8,10,8,16,4,8,2,0.15,1.0,1699659661,1699661461
617,617,2023-11-11 00:14:36.1433985,2023-11-11 00:44:36.1497108,general,random8,10,8,16,4,8,2,0.15,1.0,1699661676,1699663476
618,618,2023-11-16 21:11:58.8405660,2023-11-16 21:41:58.8429866,unl,informed9,8,6,12,4,6,2,0.15,3.0,1700169118,1700170918
619,619,2023-11-16 21:45:27.7643452,2023-11-16 22:15:27.7677002,unl,informed9,8,6,12,4,6,2,0.15,3.0,1700171127,1700172927
620,620,2023-11-16 22:19:01.4425719,2023-11-16 22:49:01.4476435,unl,informed9,8,6,12,4,6,2,0.15,3.0,1700173141,1700174941
621,621,2023-11-16 22:52:35.0247897,2023-11-16 23:22:35.0288852,unl,informed10,16,6,20,4,6,4,0.50,1.0,1700175155,1700176955
622,622,2023-11-16 23:26:08.6240216,2023-11-16 23:56:08.6271928,unl,informed10,16,6,20,4,6,4,0.50,1.0,1700177168,1700178968
623,623,2023-11-16 23:59:42.3830669,2023-11-17 00:29:42.3863504,unl,informed10,16,6,20,4,6,4,0.50,1.0,1700179182,1700180982
624,624,2023-11-17 00:33:15.8462845,2023-11-17 01:03:15.8485642,unl,informed11,16,8,20,8,8,4,0.15,3.0,1700181195,1700182995
625,625,2023-11-17 01:06:49.2800187,2023-11-17 01:36:49.2818318,unl,informed11,16,8,20,8,8,4,0.15,3.0,1700183209,1700185009


In [66]:
data = pd.read_csv('../datasets/timeSeries1s.csv', header=0,  index_col=0)
message = pd.DataFrame(data)

idx = [3.0, 17.0]
message = message[message["type"].isin(idx)]
message = message.drop(columns=["type", "identifier"])

message.head(10)

,experiment,_time,count
21,618,2023-11-16 21:12:00+00:00,136
22,618,2023-11-16 21:12:03+00:00,134
23,618,2023-11-16 21:12:06+00:00,133
24,618,2023-11-16 21:12:09+00:00,134
25,618,2023-11-16 21:12:12+00:00,134
26,618,2023-11-16 21:12:15+00:00,136
27,618,2023-11-16 21:12:18+00:00,138
28,618,2023-11-16 21:12:21+00:00,136
29,618,2023-11-16 21:12:24+00:00,135
30,618,2023-11-16 21:12:27+00:00,135


In [69]:
dfNoIndex = message.reset_index()
dfNoIndex['_time'] = pd.to_datetime(dfNoIndex["_time"], format='mixed')#.tz_localize(None)

by_time = dfNoIndex.groupby([dfNoIndex['experiment'],pd.Grouper(key="_time", freq='1s')])["count"].sum().reset_index()
# dfAggTime = by_time.rename(columns={"_measurement": "count"})
by_time.head(10)

,experiment,_time,count
0,385,2023-10-04 13:38:37+00:00,1029
1,385,2023-10-04 13:38:38+00:00,858
2,385,2023-10-04 13:38:39+00:00,802
3,385,2023-10-04 13:38:40+00:00,2451
4,385,2023-10-04 13:38:41+00:00,1198
5,385,2023-10-04 13:38:42+00:00,809
6,385,2023-10-04 13:38:43+00:00,2442
7,385,2023-10-04 13:38:44+00:00,1209
8,385,2023-10-04 13:38:45+00:00,816
9,385,2023-10-04 13:38:46+00:00,2453


In [70]:
# message["duration"] = (message["end"]-message["start"])/pd.Timedelta(seconds=1)

dfAggTime = by_time.merge(experiments, on=["experiment"], how='inner')
dfAggTime = dfAggTime.drop(columns=['parameter', 'topology', 'd', 'dhi', 'dlo', 'dscore', 'dlazy', 'dout', 'gossipFactor', 'interval', 'startUnix', 'endUnix'])

dfAggTime['_time'] = pd.to_datetime(dfAggTime["_time"], format='mixed')#.tz_localize(None)
dfAggTime['start'] = pd.to_datetime(dfAggTime["start"], format='mixed')#.tz_localize(None)
dfAggTime['end'] = pd.to_datetime(dfAggTime["end"], format='mixed')#.tz_localize(None)

dfAggTime["duration"] = (dfAggTime["end"]-dfAggTime["start"])/pd.Timedelta(seconds=1)

dfAggTime.head(10)

,experiment,_time,count,start,end,duration
0,385,2023-10-04 13:38:37+00:00,1029,2023-10-04 13:25:01.075225100,2023-10-04 13:40:01.077446200,900.002221
1,385,2023-10-04 13:38:38+00:00,858,2023-10-04 13:25:01.075225100,2023-10-04 13:40:01.077446200,900.002221
2,385,2023-10-04 13:38:39+00:00,802,2023-10-04 13:25:01.075225100,2023-10-04 13:40:01.077446200,900.002221
3,385,2023-10-04 13:38:40+00:00,2451,2023-10-04 13:25:01.075225100,2023-10-04 13:40:01.077446200,900.002221
4,385,2023-10-04 13:38:41+00:00,1198,2023-10-04 13:25:01.075225100,2023-10-04 13:40:01.077446200,900.002221
5,385,2023-10-04 13:38:42+00:00,809,2023-10-04 13:25:01.075225100,2023-10-04 13:40:01.077446200,900.002221
6,385,2023-10-04 13:38:43+00:00,2442,2023-10-04 13:25:01.075225100,2023-10-04 13:40:01.077446200,900.002221
7,385,2023-10-04 13:38:44+00:00,1209,2023-10-04 13:25:01.075225100,2023-10-04 13:40:01.077446200,900.002221
8,385,2023-10-04 13:38:45+00:00,816,2023-10-04 13:25:01.075225100,2023-10-04 13:40:01.077446200,900.002221
9,385,2023-10-04 13:38:46+00:00,2453,2023-10-04 13:25:01.075225100,2023-10-04 13:40:01.077446200,900.002221


In [42]:
experiments.loc[experiments['experiment'] == 390].head(50)

,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix
390,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518,unl,reference,8,6,12,4,8,2,0.25,1.0,1696445523,1696447323


In [76]:
df=dfAggTime.drop(columns=["start", "end", "_time"])

avgPropExp = df.loc[df["duration"] >= 1800]
avgPropExp = avgPropExp.groupby(['experiment', 'duration']).agg('sum')

avgPropExp.reset_index(inplace=True)
avgPropExp['mean'] = avgPropExp['count']/avgPropExp['duration']

avgBand = avgPropExp.drop(columns=['duration', 'count'])

avgBand.head(10)

,experiment,mean
0,390,285.933511
1,391,195.675311
2,392,86.268800
3,393,108.486362
4,394,94.085382
5,395,36.928853
6,396,79.777656
7,397,69.534879
8,398,7.107765
9,402,75.661552


In [77]:
final = experiments.drop(columns=['startUnix', 'endUnix', 'parameter'])

final.head(10)

,experiment,start,end,topology,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval
0,0,2023-08-21 12:48:22.3741823,2023-08-21 13:18:22.3952804,unl,8,6,12,4,8,2,0.25,1.0
1,1,2023-08-21 13:21:55.6321388,2023-08-21 13:51:55.6334369,unl,8,6,12,4,8,2,0.25,1.0
2,2,2023-08-21 13:55:28.9783578,2023-08-21 14:25:28.9820206,unl,8,6,12,4,8,2,0.25,1.0
3,3,2023-08-21 14:29:02.5866046,2023-08-21 14:59:02.6871797,unl,8,6,12,4,8,2,0.25,0.5
4,4,2023-08-21 15:02:35.9708968,2023-08-21 15:32:36.0272486,unl,8,6,12,4,8,2,0.25,0.5
5,5,2023-08-21 15:36:09.3380555,2023-08-21 16:06:09.3391894,unl,8,6,12,4,8,2,0.25,0.5
6,6,2023-08-21 16:09:43.1463718,2023-08-21 16:39:43.1814633,unl,8,6,12,4,8,2,0.25,30.0
7,7,2023-08-21 16:43:16.7045785,2023-08-21 17:13:16.7785160,unl,8,6,12,4,8,2,0.25,30.0
8,8,2023-08-21 17:16:50.1424767,2023-08-21 17:46:50.1924402,unl,8,6,12,4,8,2,0.25,30.0
9,9,2023-08-21 17:50:23.0310174,2023-08-21 18:20:23.0333186,unl,8,6,12,4,8,2,0.25,3.0


In [78]:
finalBandwidth = final.merge(avgBand, on='experiment')
finalBandwidth = finalBandwidth.rename(columns={'mean' : 'bandwidth'})

finalBandwidth.to_csv('bandwidth.csv')
finalBandwidth.head(10)

,experiment,start,end,topology,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,bandwidth
0,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518,unl,8,6,12,4,8,2,0.25,1.0,285.933511
1,391,2023-10-04 19:25:37.1533399,2023-10-04 19:55:37.1555856,unl,8,6,12,4,8,2,0.25,1.0,195.675311
2,392,2023-10-04 19:59:12.6589825,2023-10-04 20:29:12.6608307,unl,8,6,12,4,8,2,0.25,1.0,86.268800
3,393,2023-10-04 20:32:47.3098616,2023-10-04 21:02:47.3149178,unl,8,6,12,4,8,2,0.25,3.0,108.486362
4,394,2023-10-04 21:06:22.7755634,2023-10-04 21:36:22.7788800,unl,8,6,12,4,8,2,0.25,3.0,94.085382
5,395,2023-10-04 21:39:58.6528165,2023-10-04 22:09:58.6545536,unl,8,6,12,4,8,2,0.25,3.0,36.928853
6,396,2023-10-04 22:13:33.8146704,2023-10-04 22:43:33.8174094,unl,16,8,20,16,8,4,0.50,1.0,79.777656
7,397,2023-10-04 22:47:09.3837110,2023-10-04 23:17:09.3868442,unl,16,8,20,16,8,4,0.50,1.0,69.534879
8,398,2023-10-04 23:20:44.6001719,2023-10-04 23:50:44.6033804,unl,16,8,20,16,8,4,0.50,1.0,7.107765
9,402,2023-10-05 01:35:05.2471450,2023-10-05 02:05:05.2498822,unl,16,8,20,16,8,4,0.50,0.5,75.661552
